# Diagnosing SSL Issues in EdgarTools

If you're getting SSL certificate errors when using EdgarTools, this notebook will help you diagnose the problem and find a solution.

## Common SSL Error Messages

You might see errors like:
- `SSLCertVerificationError: certificate verify failed`
- `CERTIFICATE_VERIFY_FAILED`
- `unable to get local issuer certificate`

These typically occur in **corporate networks** that use SSL inspection proxies, or in environments with custom certificate configurations.

## Step 1: Run the Diagnostic Tool

EdgarTools includes a built-in SSL diagnostic tool that checks your connection to SEC.gov:

In [ ]:
from edgar import diagnose_ssl

# Run diagnostics and display results
result = diagnose_ssl()

## Understanding the Results

The diagnostic runs 5 tests:

| Test | What it checks |
|------|---------------|
| **1. DNS Resolution** | Can your computer find SEC.gov's IP address? |
| **2. TCP Connection** | Can you connect to SEC.gov on port 443? |
| **3. SSL Handshake** | Does the SSL certificate verify correctly? |
| **4. HTTP (default SSL)** | Can you make an HTTPS request with standard settings? |
| **5. HTTP (your settings)** | Can you make an HTTPS request with your configured settings? |

### Key Insight: Tests 4 vs 5

- **Test 4** uses default SSL verification (like a fresh Python install)
- **Test 5** uses YOUR configured settings (e.g., if you've called `configure_http(verify_ssl=False)`)

If Test 4 fails but Test 5 passes, your workaround is working!

## Step 2: Check the Results Programmatically

You can inspect the diagnostic result object to understand what's happening:

In [ ]:
# Get the result without displaying (useful for scripting)
result = diagnose_ssl(display=False)

# Check if SSL is working
print(f"SSL OK: {result.ssl_ok}")
print(f"Diagnosis: {result.diagnosis}")

In [ ]:
# Check specific test results
net = result.network_tests

print(f"DNS resolved: {net.dns_resolved} (IP: {net.dns_ip})")
print(f"TCP connected: {net.tcp_connected}")
print(f"SSL handshake success: {net.ssl_handshake.success if net.ssl_handshake else 'N/A'}")
print(f"HTTP with default SSL: {net.http_request_ok}")
print(f"HTTP with your settings: {net.configured_http_ok}")

In [ ]:
# Check if a corporate proxy was detected
if result.network_tests.ssl_handshake:
    if result.network_tests.ssl_handshake.is_corporate_proxy:
        print("Corporate SSL inspection proxy detected!")
        print("Your network intercepts HTTPS traffic and replaces certificates.")
    else:
        print("No corporate proxy detected.")

## Step 3: Apply the Fix

Based on the diagnosis, here are the common solutions:

### Solution A: Disable SSL Verification (Quick Fix)

If you're on a corporate network with SSL inspection, this is often the easiest solution:

In [ ]:
from edgar import configure_http

# Disable SSL verification
configure_http(verify_ssl=False)

# Now test if it works
from edgar import Company

apple = Company("AAPL")
print(f"Success! Got {apple.name}")

### Solution B: Add Corporate CA Certificate (More Secure)

If your IT department can provide your organization's root CA certificate:

```python
import os
os.environ["REQUESTS_CA_BUNDLE"] = "/path/to/corporate-ca.pem"

# Then import and use edgar normally
from edgar import Company
```

Or set it as an environment variable before running Python:
```bash
export REQUESTS_CA_BUNDLE="/path/to/corporate-ca.pem"
python your_script.py
```

## Step 4: Check Recommendations

The diagnostic tool provides specific recommendations based on what it finds:

In [ ]:
result = diagnose_ssl(display=False)

if result.recommendations:
    print("Recommendations:")
    for i, rec in enumerate(result.recommendations, 1):
        print(f"\n{i}. {rec.title}")
        print(f"   {rec.description}")
        if rec.code_snippet:
            print(f"\n   Code:\n   {rec.code_snippet[:100]}...")
else:
    print("No issues found - SSL is working correctly!")

## Bonus: Environment Information

You can also inspect the environment details that were gathered:

In [ ]:
result = diagnose_ssl(display=False)

env = result.environment
print(f"Python: {env.python_version}")
print(f"Platform: {env.platform}")
print(f"EdgarTools: {env.edgartools_version}")
print(f"httpx: {env.httpx_version}")
print(f"certifi: {env.certifi_version}")
print(f"cryptography: {env.cryptography_version or 'Not installed'}")

In [ ]:
# Check certificate and proxy configuration
cert = result.certificate_config
print("Certificate Configuration:")
print(f"  REQUESTS_CA_BUNDLE: {cert.requests_ca_bundle or 'Not set'}")
print(f"  SSL_CERT_FILE: {cert.ssl_cert_file or 'Not set'}")
print(f"  certifi bundle: {cert.certifi_path}")

if result.proxy_config:
    proxy = result.proxy_config
    print("\nProxy Configuration:")
    print(f"  HTTP_PROXY: {proxy.http_proxy or 'Not set'}")
    print(f"  HTTPS_PROXY: {proxy.https_proxy or 'Not set'}")

## Command Line Usage

You can also run the diagnostic from the command line:

```bash
python -m edgar.diagnose_ssl
```

This is useful for troubleshooting in environments where Jupyter isn't available.

## Need More Help?

- **Documentation**: [SSL Verification Guide](https://github.com/dgunning/edgartools/blob/main/docs/guides/ssl_verification.md)
- **GitHub Issues**: [Report a problem](https://github.com/dgunning/edgartools/issues)
- **Common causes**: Corporate firewalls, VPNs, outdated certifi package